<a href="https://colab.research.google.com/github/tuchinhung/Quantu-ML/blob/main/Comparacao_do_modelo_Classico_VS_Quantico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%%capture 

# Comment this out if you don't want to install pennylane from this notebook
!pip install pennylane
# Comment this out if you don't want to install torch from this notebook
!pip install torch
# Comment this out if you don't want to install matplotlib from this notebook
!pip install matplotlib

Modelo Classico regressao linear

In [24]:
import torch
from torch.autograd import Variable
import numpy as np

np.random.seed(42)

data = torch.tensor([(0., 0.), (0.1, 0.1), (0.2, 0.2)])
print(data)

tensor([[0.0000, 0.0000],
        [0.1000, 0.1000],
        [0.2000, 0.2000]])


In [25]:
def model(phi, x=None):
  return x*phi

In [26]:
# Função de custo (a-b)^2 Perda de erro quadrático
def loss(a, b):
    return torch.abs(a - b)**2

# Perda média
def average_loss(phi):
    c = 0
    for x, y in data:
        c += loss(model(phi, x=x),  y) # Função de perda de erro quadrático
    return c

phi_new = Variable(torch.tensor(0.1), requires_grad=True)
print(phi_new)
opt = torch.optim.Adam([phi_new], lr=0.02)
print(opt)

tensor(0.1000, requires_grad=True)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.02
    weight_decay: 0
)


In [27]:
for i in range(10):
  loss1 = average_loss(phi_new)
  print(i, loss1.item())
  loss1.backward()
  opt.step()

0 0.04050000011920929
1 0.03872000053524971
2 0.03703787177801132
3 0.03540395200252533
4 0.03380205109715462
5 0.03222564607858658
6 0.030672047287225723
7 0.029140427708625793
8 0.027630960568785667
9 0.02614441327750683


Modelo Quantico Circuito

In [28]:
from pennylane import *
import torch
from torch.autograd import Variable
import numpy as np
np.random.seed(42)

data1 = [(0., 0.), (0.1, 0.1), (0.2, 0.2)]
print(data1)

[(0.0, 0.0), (0.1, 0.1), (0.2, 0.2)]


In [29]:
# 'default.qubit': um simulador de estado simples de arquiteturas de circuitos quânticos baseadas em qubit.
# com 2 qubit
dev = device('default.qubit', wires=2)

@qnode(dev, interface='torch')
def circuit_model(phi, x=None):

    templates.AngleEmbedding(features=[x], wires=[0])
    templates.BasicEntanglerLayers(weights=phi, wires=[0, 1])
    return expval(PauliZ(wires=1)) # Valor de expectativa do observável fornecido.
    # o circuito a seguir retorna o valor esperado do operador Pauli Z observável no fio 1


In [30]:
# Função de custo (a-b)^2 Perda de erro quadrático
def loss(a, b):
    return torch.abs(a - b)**2

# Perda média
def average_loss(phi):
    c = 0
    for x, y in data:
        c += loss(circuit_model(phi, x=x),  y) # Função de perda de erro quadrático
    return c

phi_new = Variable(torch.tensor([[0.1, 0.2], [-0.5, 0.1]]), requires_grad=True)
print(phi_new)
opt = torch.optim.Adam([phi_new], lr=0.02)
print(opt)

tensor([[ 0.1000,  0.2000],
        [-0.5000,  0.1000]], requires_grad=True)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.02
    weight_decay: 0
)


In [31]:
for i in range(10):
  loss1 = average_loss(phi_new)
  print(i, loss1.item())
  loss1.backward()
  opt.step()

0 1.65564449517298
1 1.567177708193579
2 1.4789048264364286
3 1.3891743567633288
4 1.2978172022632155
5 1.2051195848780754
6 1.1115742469587113
7 1.0177922191821251
8 0.9244605097013131
9 0.8323157438655978
